In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import gensim,os,sklearn,tqdm,nltk

from collections import Counter
from datetime import datetime


# 1. load data
### counter

In [2]:
counter = Counter()

counter_dir="D:/work/fortune_street/002 news_analyze/002 data/003 outcome_data/word_counter_data/"
txt_dir="D:/work/fortune_street/002 news_analyze/002 data/002 corpus_data/token_txt_data/"

list_counter_filenames=list(os.walk(counter_dir))[0][2]
print(list_counter_filenames)
list_txt_source=list(os.walk(txt_dir))[0][1]
print(list_txt_source)

for counter_file in list_counter_filenames:
    if counter_file.startswith("word_counter"):
        print(counter_dir+counter_file)
        current_counter=np.load(counter_dir+counter_file)
        counter.update(current_counter.tolist())
counter.most_common(10)

['word_counter.npy', 'word_counter_2019_02_08.npy', 'word_counter_2019_02_12.npy']
['cnbc_news_data', 'oil_price_news_data', 'uk_reuters_news_data']
D:/work/fortune_street/002 news_analyze/002 data/003 outcome_data/word_counter_data/word_counter.npy
D:/work/fortune_street/002 news_analyze/002 data/003 outcome_data/word_counter_data/word_counter_2019_02_08.npy
D:/work/fortune_street/002 news_analyze/002 data/003 outcome_data/word_counter_data/word_counter_2019_02_12.npy


[('the', 551201),
 ('to', 300006),
 ('in', 245871),
 ('of', 242474),
 ('and', 228839),
 ('a', 215503),
 ('on', 125068),
 ('s', 115873),
 ('by', 114743),
 ('said', 97069)]

### load news data

In [45]:
# 'CC'表示并列连词，'RB'表示副词，'IN'表示介词，'NN'表示名词，'JJ'表示形容词
def pos_filter(aListToken):
    remove_pos_list=['IN','DT','FW','CC','TO','MD','WP','LS','EX','PDT','RP','PRP$']
    text1=aListToken
    if '' in text1:
        text1.remove('')
    pos_list=nltk.pos_tag(text1)
    return([word for word,pos in pos_list if pos not in remove_pos_list])

In [68]:
def read_news_txt_file(aFilepath):
    token_list=[]
    with open(aFilepath,"r",encoding='utf8') as f:
        news_title=f.readline()
        line = news_title
        while line :
            line = f.readline()
            token_list.append(line.strip())
    return(news_title,token_list)

full_list=[]
for aSource in list_txt_source:
    for subpath , null , list_news_filenames in tqdm.tqdm(list(os.walk(txt_dir+aSource))[1:]):
        news_publish_date=subpath.split("\\")[1]
        source=subpath.split("\\")[0].split("/")[-1]
        for aFilename in list_news_filenames:
            aFilepath="/".join([subpath, aFilename])
            news_title,news_tokens_list=(read_news_txt_file(aFilepath))
            
            ##apply pos filter and save
            pos_filtered_Filepath=aFilepath.replace('token_txt_data','token_pos_filtered_txt_data')
            pos_filtered_dir='/'.join(pos_filtered_Filepath.split('/')[:-1])
            if not os.path.exists(pos_filtered_dir):
                os.makedirs(pos_filtered_dir)
            filtered=pos_filter(news_tokens_list)
            a_title,a_tokens=news_title,filtered
            with open(pos_filtered_Filepath, "w",encoding="utf8") as text_file:
                text_file.writelines(a_title.lower()+"\n")
                for token in a_tokens:
                    text_file.write(token+"\n")
                    
            ##append to full list
            out_dict={'news_publish_date':news_publish_date,
                      'news_title':news_title,
                      'news_tokens_list':news_tokens_list,
                      'source':source,
                      'news_pos_filtered_tokens_list':filtered}
            full_list.append(out_dict)
            
full_news_df=pd.DataFrame(full_list)
full_news_df.head(10)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 360/360 [06:45<00:00,  1.04it/s]


,news_pos_filtered_tokens_list,news_publish_date,news_title,news_tokens_list,source
0,"[singapore, dec, reuters, oil, prices, slumped...",2018-12-04,update 3-oil prices skid with global stock mar...,"[singapore, dec, reuters, oil, prices, slumped...",cnbc_news_data
1,"[singapore, dec, reuters, oil, prices, slipped...",2018-12-04,u.s. oil prices slide with global markets on e...,"[singapore, dec, reuters, oil, prices, slipped...",cnbc_news_data
2,"[heating, oil, futures, turn, negative, trade,...",2018-12-04,heating oil futures turn negative in post-sett...,"[heating, oil, futures, turn, negative, in, tr...",cnbc_news_data
3,"[oil, futures, fall, trade, api, data, show, s...",2018-12-04,oil futures fall in post-settlement trade afte...,"[oil, futures, fall, in, trade, after, api, da...",cnbc_news_data
4,"[opec, meeting, singapore, dec, reuters, oil, ...",2018-12-05,update 1-oil prices dip as stock markets slide...,"[opec, meeting, singapore, dec, reuters, oil, ...",cnbc_news_data
5,"[singapore, dec, reuters, oil, prices, dipped,...",2018-12-05,oil prices ease in cautious trading ahead of o...,"[singapore, dec, reuters, oil, prices, dipped,...",cnbc_news_data
6,"[oil, prices, extend, gains, brent, crude, mor...",2018-12-05,"oil prices extend gains, brent crude up by mor...","[oil, prices, extend, gains, brent, crude, up,...",cnbc_news_data
7,"[singapore, dec, reuters, oil, prices, fell, w...",2018-12-05,update 5-oil prices fall on global growth conc...,"[singapore, dec, reuters, oil, prices, fell, o...",cnbc_news_data
8,"[singapore, dec, reuters, oil, prices, fell, w...",2018-12-05,update 4-oil prices slip on global growth conc...,"[singapore, dec, reuters, oil, prices, fell, o...",cnbc_news_data
9,"[decision, singapore, dec, reuters, oil, price...",2018-12-06,"oil firms on drop in us crude stocks, market a...","[decision, singapore, dec, reuters, oil, price...",cnbc_news_data


In [69]:
full_news_df.news_pos_filtered_tokens_list

0        [singapore, dec, reuters, oil, prices, slumped...
1        [singapore, dec, reuters, oil, prices, slipped...
2        [heating, oil, futures, turn, negative, trade,...
3        [oil, futures, fall, trade, api, data, show, s...
4        [opec, meeting, singapore, dec, reuters, oil, ...
5        [singapore, dec, reuters, oil, prices, dipped,...
6        [oil, prices, extend, gains, brent, crude, mor...
7        [singapore, dec, reuters, oil, prices, fell, w...
8        [singapore, dec, reuters, oil, prices, fell, w...
9        [decision, singapore, dec, reuters, oil, price...
10       [vienna, dec, reuters, opec, tentatively, agre...
11       [slide, london, dec, reuters, yields, german, ...
12       [vienna, dec, reuters, opec, tentatively, agre...
13       [vienna, dec, reuters, opec, tentatively, agre...
14       [brent, crude, oil, futures, fall, pct, wti, f...
15       [crude, futures, extend, drop, falling, day, b...
16       [meeting, singapore, dec, reuters, oil, prices.